<a href="https://colab.research.google.com/github/eugeniusz-raspberrypi/Export_model_cpk_to_onnx/blob/main/Export_model_cpk_to_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Instalacja zależności (wykonaj raz)

In [ ]:
# KOMÓRKA 0 – Instalacja zależności (wykonaj raz)
!pip install -q piper-tts onnx onnxruntime torch soundfile numpy tqdm
# (piper-tts już zawiera wszystkie potrzebne części train/*)

1. Wgranie Twojego modelu .cpk i config.json

In [ ]:
# KOMÓRKA 1 – Wgranie Twojego modelu .cpk i config.json
from google.colab import files
uploaded = files.upload()   # w tym miejscu wczytaj swój plik .cpk i (opcjonalnie) config.json

2. Sam skrypt (wersja Colab-friendly) – wystarczy uruchomić raz i wpisać nazwy plików

In [ ]:
# KOMÓRKA 2 – Sam skrypt (wersja Colab-friendly) – wystarczy uruchomić raz i wpisać nazwy plików
import os
import json
import torch
from piper.train.checkpoint import load_checkpoint
from piper.voice import PiperVoice

# ——— WPISZ TUTAJ NAZWY SWOICH PLIKÓW ———
cpk_file     = "moj_glos.cpk"          # ← zmień na nazwę Twojego .cpk
config_file  = "config.json"           # ← jeśli masz osobny config.json, w przeciwnym razie zostaw
output_name  = "moj_glos_polski"       # ← jak ma się nazywać finalny model (bez rozszerzenia)

# Automatyczne wykrycie config.json jeśli nie podałeś
if not os.path.exists(config_file):
    potential = os.path.join(os.path.dirname(cpk_file), "config.json")
    if os.path.exists(potential):
        config_file = potential

# ——— Reszta skryptu (nie ruszaj) ———
print(f"Używany .cpk: {cpk_file}")
print(f"Używany config: {config_file}")

model_dir = os.path.dirname(cpk_file) if os.path.dirname(cpk_file) else "."
model = load_checkpoint(cpk_path=cpk_file, model_dir=model_dir, use_cuda=False)
voice = PiperVoice(model, config_path=config_file)

voice.model.eval()
print("Eksportowanie do ONNX...")

dummy_text = "To jest testowy tekst do wygenerowania wejścia."
synthesize_input = voice.create_synthesize_input(dummy_text)

input_ids = torch.tensor(synthesize_input["input_ids"], dtype=torch.long).unsqueeze(0)
scales = torch.tensor([1.0, 0.667, 0.8], dtype=torch.float32)

export_kwargs = {"input_ids": input_ids, "scales": scales}
input_names = ["input_ids", "scales"]

if voice.config.num_speakers > 1:
    speaker_id = torch.tensor([0], dtype=torch.long)
    export_kwargs["speaker_id"] = speaker_id
    input_names.append("speaker_id")

onnx_path = f"{output_name}.onnx"
torch.onnx.export(
    voice.model,
    args=tuple(export_kwargs.values()),
    f=onnx_path,
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    input_names=input_names,
    output_names=["audio"],
    dynamic_axes={"input_ids": {0: "batch", 1: "sequence"}},
    verbose=False
)

# Generowanie .onnx.json (metadata)
json_path = f"{output_name}.onnx.json"
with open(config_file) as f:
    config_data = json.load(f)

# Dodanie brakujących pól inferencji (standard Piper)
config_data.setdefault("inference", {
    "noise_scale": 0.667,
    "length_scale": 1.0,
    "noise_w": 0.8
})
config_data["name"] = output_name
config_data["key"]  = output_name

with open(json_path, "w") as f:
    json.dump(config_data, f, indent=4)

print("GOTOWE!")
print(f" • {onnx_path}  → {os.path.getsize(onnx_path)/1024/1024:.1f} MB")
print(f" • {json_path}")

# Pobieranie na swój komputer
from google.colab import files
files.download(onnx_path)
files.download(json_path)

3. (opcjonalna) szybki test czy model działa

In [ ]:
# KOMÓRKA 3 (opcjonalna) – szybki test czy model działa

# Zainstaluj binarkę piper (najnowszą)
!wget -q https://github.com/rhasspy/piper/releases/download/v1.2.0/piper_amd64.tar.gz
!tar -xf piper_amd64.tar.gz
!chmod +x piper/piper

# Test
!./piper/piper --model {output_name}.onnx --output_file test.wav --text "Cześć, to jest mój sklonowany głos w języku polskim. Działa idealnie!"
files.download("test.wav")